https://kevinzakka.github.io/2017/01/18/stn-part2/

### Pooling Layers

It turns out that the pooling layers we use in our neural network architectures actually endow our models with a certain degree of spatial invariance. Recall that the pooling operator acts as a sort of downsampling mechanism. It progressively reduces the spatial size of the feature map along the depth dimension, cutting down the amount of parameters and computational cost.

<img src='./imgs/pool.png'>


##### How exactly does it provide invariance? 

The idea behind pooling is to take a complex input, split it up into cells, and “pool” the information from these complex cells to produce a set of simpler cells that describe the output. So for example, say we have 3 images of the number 7, each in a different orientation. ***A pool over a small grid in each image would detect the number 7 regardless of its position in that grid since we’d be capturing approximately the same information by aggregating pixel values.***

> ?

Now there are a few downsides to pooling which make it an undesirable operator. For one, pooling is destructive. It discards 75% of feature activations when it is used, meaning we are guaranteed to lose exact **positional information**. Now you may be wondering why this is bad since we mentioned earlier that it endowed our network with some spatial robustness. Well the thing is that positional information is invaluable in visual recognition tasks. Think of our cat classifier above. It may be important to know where the position of the whiskers are relative to, say the snout. 

Another limitation of pooling is that it is local and predefined. ***With a small receptive field, the effects of a pooling operator are only felt towards deeper layers of the network meaning intermediate feature maps may suffer from large input distortions. And remember, we can’t just increase the receptive field arbitrarily because then that would downsample our feature map too agressively.***

> ?

The main takeaway is that ConvNets are not invariant to relatively large input distortions. This limitation is due to having only a restricted, pre-defined pooling mechanism for dealing with spatial variation of the data. This is where Spatial Transformer Networks come into play!

### Spatial Transformer Networks (STNs)

The Spatial Transformer mechanism addresses the issues above by providing Convolutional Neural Networks with explicit spatial transformation capabilities. It possesses 3 defining properties that make it very appealing.

* modular: STNs can be inserted anywhere into existing architectures with relatively small tweaking.
* differentiable: STNs can be trained with backprop allowing for end-to-end training of the models they are injected in.
* dynamic: STNs perform active spatial transformation on a feature map for each input sample as compared to the pooling layer which acted identically for all input samples.

As you can see, the Spatial Transformer is superior to the Pooling operator in all regards. So this begs the following question: what exactly is a Spatial Transformer?

<img src='./imgs/arch.png'>


The Spatial Transformer module consists in three components shown in the figure above: 

* a localisation network
* a grid generator
* a sampler

One very very important note before we jump into the code!<br/>
An image processing affine transformation usually follows the 3-tstep pipelin below:

* First, we create sampling grid composedof $(x, y)$ coordinates. For example, given a 400x400 grayscale image, we create a meshgrid of same dimension, that is, evenly spaced $x \in [0, W] and y \in [0, H]$
* We then apply the transformation matrix to the sampling grid generated in the step above.
* Finally, we sample the resulting grid from the original image using the desired interpolation technique.


Recall that we can’t just blindly rush to the input image and apply our affine transformation. It’s important to first create a sampling grid, transform it, and then sample the input image using the grid. With that being said, let’s jump into the core components of the Spatial Transformer.




### Localisation Network

The goal of the localisation network is to spit out the parameters $\theta$ of the affine transformation that’ll be applied to the input feature map. More formally, our localisation net is defined as follows:

* input: feature map U of shape (H, W, C)
* output: transformation matrix θ of shape (6,)
* architecture: fully-connected network or ConvNet as well.

As we train our network, we would like our localisation net to output more and more accurate thetas. What do we mean by accurate? Well, think of our digit 7 rotated by 90 degrees counterclockwise. After say 2 epochs, our localisation net may output a transformation matrix which performs a 45 degree clockwise rotation and after 5 epochs for example, it may actually learn to do a complete 90 degree clockwise rotation. The effect is that our output image looks like a standard digit 7, something our neural network has seen in the training data and can easily classify.

Another way to look at it is that the localisation network learns to store the knowledge of how to transform each training sample in the weights of its layers.

### Parametrised Sampling Grid

The grid generator’s job is to output a parametrised sampling grid, which is a set of points where the input map should be sampled to produce the desired transformed output.

Concretely, the grid generator first creates a normalized meshgrid of the same size as the input image U of shape (H, W), that is, a set of indices (xt,yt) that cover the whole input feature map (the subscript t here stands for target coordinates in the output feature map). Then, since we’re applying an affine transformation to this grid and would like to use translations, we proceed by adding a row of ones to our coordinate vector to obtain its homogeneous equivalent. This is the little trick we also talked about last week. Finally, we reshape our 6 parameter θ to a 2x3 matrix and perform the following multiplication which results in our desired parametrised sampling grid.

$$\begin{bmatrix}
x^s\\ y^s\end{bmatrix} =  \begin{bmatrix}
\theta_{11} & \theta_{12} & \theta_{13} \\
\theta_{21} & \theta_{22} & \theta_{23} 
\end{bmatrix} \begin{bmatrix}
x^t \\ y^t \\ 1
\end{bmatrix}
$$

The column vector $\begin{bmatrix} x_s \\ y_s \end{bmatrix}$ consists in a set of indices that tell us where we should sample our input to obtain the desired transformed output.

But wait a minute, what if those indices are fractional? Bingo! That’s why we learned about bilinear interpolation and this is exactly what we do next.